In [46]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_keys
import time
from datetime import datetime

In [47]:
# Create a set of random latitude and longitude combinations.

lats = np.random.uniform(low=-90.000, high=90.000, size=8500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=8500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [48]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in lat_lngs:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

2043

In [49]:
#Create base_url

url = "https://api.openweathermap.org/data/2.5/weather?units=Imperial&appid=" + weather_api_keys


In [55]:
# Create an empty list to hold the weather data.
city_data = []

# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in our list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        
        city_weather = requests.get(city_url).json()

        # # Parse out the needed data.
        
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        city_description = city_weather["weather"][0]["description"]

        # Append the city information into city_data list.
        
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Description": city_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | vao
Processing Record 2 of Set 1 | busselton
Processing Record 3 of Set 1 | hilo
Processing Record 4 of Set 1 | betanzos
Processing Record 5 of Set 1 | uglovskoye
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | mataura
Processing Record 8 of Set 1 | bluff
Processing Record 9 of Set 1 | longyearbyen
Processing Record 10 of Set 1 | casper
Processing Record 11 of Set 1 | ahipara
Processing Record 12 of Set 1 | atambua
Processing Record 13 of Set 1 | constantine
Processing Record 14 of Set 1 | mastic beach
Processing Record 15 of Set 1 | barrow
Processing Record 16 of Set 1 | tura
Processing Record 17 of Set 1 | kaitangata
Processing Record 18 of Set 1 | punta arenas
Processing Record 19 of Set 1 | halifax
Processing Record 20 of Set 1 | preston
Processing Record 21 of Set 1 | coihaique
Processing Record 22 of Set 1 | fortuna
Processing Record 23 of Set 1 | upernavik
Processing 

In [57]:
# Convert the array of dictionaries to a Pandas DataFrame
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,Vao,NC,-22.6667,167.4833,69.62,62,99,11.74,overcast clouds
1,Busselton,AU,-33.6500,115.3333,58.51,71,100,15.70,overcast clouds
2,Hilo,US,19.7297,-155.0900,88.29,72,40,11.50,scattered clouds
3,Betanzos,ES,43.2804,-8.2147,66.85,86,40,2.30,scattered clouds
4,Uglovskoye,RU,51.3557,80.1942,64.60,69,28,6.33,scattered clouds
...,...,...,...,...,...,...,...,...,...
1878,Abu Kamal,SY,34.4506,40.9171,88.84,15,0,2.53,clear sky
1879,Blois,FR,47.5833,1.3333,68.38,42,70,6.78,broken clouds
1880,Keelung,TW,25.1283,121.7419,88.05,78,3,5.82,clear sky
1881,Soe,ID,-9.8607,124.2840,61.48,82,92,5.79,overcast clouds


In [58]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")